In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import glob 
import matplotlib.pyplot as plt
import matplotlib as mpl

sns.set_context("talk")
data_dir = "/home/luisasantus/Desktop/crg_cluster/projects/structural_regression/results/homfam/"

# load families summary
summary_homfam_exthomfam = pd.read_csv("/home/luisasantus/Desktop/crg_cluster/data/structural_regression/stats/01_summary_homfam_exthomfam.csv")
summary_homfam = summary_homfam_exthomfam[summary_homfam_exthomfam.dataset == "homfam"]
families = summary_homfam.drop(["subset", "file", "min_length", "max_length", "perc_ref_total", "dataset"], axis = 1)

# 1. PREP SCORES
def get_scores(score_files): 
    def parse_score(score_file): 
        return(pd.read_csv(score_file,sep = ";", header = None).drop(4, axis = 1))

    scores = pd.concat(list(map(parse_score, score_files)))
    scores = scores.iloc[:,0:4]
    scores.columns = ['name', 'sp', 'tc', 'column']
    #scores.name = scores.name.str.split(".", expand = True)[:-1]
    #scores[["family", "method", "bucket_size", "align_method", "tree_method"]] = scores.name.str.split(".", expand = True)
    #scores = scores.drop("method", axis = 1)
    return(scores)


def plot_scatter_perc(df1,df2,xlabel,ylabel,
                      palette = sns.dark_palette("#3399FF", reverse = True, as_cmap=True),
                      log = True, 
                      title = "regressive on homfam", hue_var = "sim", metric = "sp", size_fig = 1): 
    f, ax = plt.subplots(figsize=(8*size_fig,6.4*size_fig ))
    
    # Prep df 
    df = df1.merge(df2, on = ["family", hue_var])
    
    
    hue = df[hue_var]
    # colorbar 
    norm = mpl.colors.Normalize( vmin=0, vmax=100)
    sm = plt.cm.ScalarMappable(cmap=palette, norm=norm)
    sm.set_array([])
    
# Color bar 
    cbar =ax.figure.colorbar(sm, ticks=[0,50,100], format=mpl.ticker.ScalarFormatter())
    cbar.ax.set_yticklabels(["0", "50", "100"]) 
    cbar.ax.set_ylabel('avg % similarity', rotation=270, labelpad = 20, fontsize = "small")
    
    metric_x = metric+"_x"
    metric_y = metric+"_y"
    # Plot 
    ax = sns.scatterplot(data = df, x = metric_x,
                    y = metric_y,
                    hue = hue_var,
                    s = 100,
                    palette = palette)

    # % above the line
    perc_y_better_than_x = (len(list(filter(lambda ele: ele == True, list(df[metric_x] <= df[metric_y])))) / len(list(df[metric_x]  >= df[metric_y] ))) * 100
    
    # Diagonal line
    ax.axline((1, 1), slope=1, ls="--", c=".2", lw = 0.8)
    
    ax.set(xlim = (0,100), ylim = (0,100))
    ax.get_legend().remove()
    
    # Axis labels
    ax.set(xlabel=xlabel,
           ylabel=ylabel,
           title = title + "\n metric: "+metric+"\n (n = "+str(len(df[metric_x] ))+") \n\n % y >= x  "+str(round(perc_y_better_than_x,1))+" \n")

In [ ]:
# 1. SEQUENCE SCORES 
sequence_scores = get_scores(glob.glob(os.path.join(data_dir, "evaluation/score/*progressive.PROBAPAIR.MBED*")))
sequence_scores[["family", "method", "align_method", "tree_method"]] = sequence_scores.name.str.split(".", expand = True)
sequence_scores = pd.merge(sequence_scores, families, on = "family")
# 2. FOLDSEEK SCORES 
#foldseek_scores = get_scores(glob.glob(os.path.join(data_dir, "evaluation/score/*foldseek*")))
# here remember to remove the scores of foldseek_sequence

# 3. FOLDSEEK_SEQUENCE
foldseek_sequence_scores = get_scores(glob.glob(os.path.join(data_dir, "evaluation/score/*foldseek_sequence*")))
foldseek_sequence_scores[["family", "align_method", "tree_method", "library_method"]] = foldseek_sequence_scores.name.str.split(".", expand = True)
foldseek_sequence_scores = pd.merge(foldseek_sequence_scores, families, on = "family")

# 4. 3DCOFFEE STRUCTURE 
structures_3dcoffee_scores = get_scores(glob.glob(os.path.join(data_dir, "evaluation/score/*3DCOFFEE.MBED*")))
structures_3dcoffee_scores[["family", "method", "align_method", "tree_method"]] = structures_3dcoffee_scores.name.str.split(".", expand = True)
structures_3dcoffee_scores = pd.merge(structures_3dcoffee_scores, families, on = "family")

In [2]:
sequence_scores = get_scores(glob.glob(os.path.join(data_dir, "evaluation/score/*progressive.PROBAPAIR.MBED*")))


In [3]:
sequence_scores

,name,sp,tc,column
0,peroxidase-ref.progressive.PROBAPAIR.MBED,89.3,65.9,70.8
0,cytb-ref.progressive.PROBAPAIR.MBED,92.5,75.0,84.3
0,subt-ref.progressive.PROBAPAIR.MBED,76.8,27.6,35.7
0,hr-ref.progressive.PROBAPAIR.MBED,99.1,93.7,98.6
0,rvp-ref.progressive.PROBAPAIR.MBED,83.6,57.7,65.0
...,...,...,...,...
0,oxidored_q6-ref.progressive.PROBAPAIR.MBED,98.8,95.5,97.1
0,blm-ref.progressive.PROBAPAIR.MBED,97.6,90.3,93.3
0,icd-ref.progressive.PROBAPAIR.MBED,94.8,84.8,87.6
0,hom-ref.progressive.PROBAPAIR.MBED,96.5,73.9,88.3


In [5]:
sequence_scores = get_scores(glob.glob(os.path.join(data_dir, "evaluation/score/*progressive*")))
sequence_scores

,name,sp,tc,column
0,bowman-ref.progressive.PROBAPAIR.FAMSA-medoid,97.7,90.0,95.9
0,mofe-ref.progressive.EXPRESSO.MBED,75.8,54.3,59.6
0,rub.progressive.FAMSA.MBED,95.9,90.7,89.8
0,ltn-ref.progressive.3DCOFFEEEXPERIMENTAL.MBED,96.9,79.8,86.8
0,ghf1.progressive.FAMSA.FAMSA-medoid,89.8,65.2,73.6
...,...,...,...,...
0,Ald_Xan_dh_2-ref.progressive.PROBAPAIR.MBED,88.1,69.7,77.1
0,hpr-ref.progressive.PROBAPAIR.FAMSA-medoid,98.3,94.3,96.3
0,sdr-ref.progressive.EXPRESSOREAL.MBED,89.2,49.8,64.7
0,ltn-ref.progressive.3DCOFFEE.FAMSA-medoid,96.7,82.2,88.7


In [6]:
sequence_scores[["family", "method", "align_method", "tree_method"]] = sequence_scores.name.str.split(".", expand = True)


In [8]:
sequence_scores[sequence_scores.family == "bowman"] 

,name,sp,tc,column,family,method,align_method,tree_method
0,bowman.progressive.FAMSA.MBED,87.5,71.4,74.4,bowman,progressive,FAMSA,MBED
0,bowman.progressive.FAMSA.FAMSA-medoid,87.5,71.4,74.4,bowman,progressive,FAMSA,FAMSA-medoid
